In [1]:
import datetime
import os
from io import StringIO

import pandas as pd
import requests
import tqdm

import algotrading_v40.connectors.zerodha as cz
import algotrading_v40.structures.date_range as sdr
import algotrading_v40.structures.instrument_desc as sid
import algotrading_v40.utils.data_nav as udn
import algotrading_v40.utils.zerodha as uz


In [2]:
def get_listing_date_for_symbol() -> dict[str, datetime.date]:
  url = "https://nsearchives.nseindia.com/content/equities/EQUITY_L.csv"
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
  }
  response = requests.get(url, headers=headers)
  df_all_eq = pd.read_csv(StringIO(response.text))
  df_all_eq.columns = df_all_eq.columns.str.strip().str.lower()
  df_all_eq.columns = df_all_eq.columns.str.replace(" ", "_")
  df_all_eq = df_all_eq[df_all_eq["series"] == "EQ"]
  return dict(
    zip(
      df_all_eq["symbol"],
      pd.to_datetime(df_all_eq["date_of_listing"], format="%d-%b-%Y").dt.date,  # type: ignore
    )
  )


def get_nifty_50_info() -> pd.DataFrame:
  url = "https://www.niftyindices.com/IndexConstituent/ind_nifty50list.csv"
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
  }
  response = requests.get(url, headers=headers)
  df_nifty_50_info = pd.read_csv(StringIO(response.text))
  df_nifty_50_info.columns = df_nifty_50_info.columns.str.strip().str.lower()
  df_nifty_50_info.columns = df_nifty_50_info.columns.str.replace(" ", "_")
  df_nifty_50_info = (
    df_nifty_50_info.drop(columns=["isin_code", "series"])
    .sort_values(by=["industry", "symbol"])
    .reset_index(drop=True)
  )
  df_nifty_50_info["type"] = "equity"
  return df_nifty_50_info


In [3]:
df_all = get_nifty_50_info()
listing_date_for_symbol = get_listing_date_for_symbol()
df_all["listing_date"] = df_all["symbol"].apply(lambda x: listing_date_for_symbol[x])
df_all

,company_name,industry,symbol,type,listing_date
0,Bajaj Auto Ltd.,Automobile and Auto Components,BAJAJ-AUTO,equity,2008-05-26
1,Eicher Motors Ltd.,Automobile and Auto Components,EICHERMOT,equity,2004-09-07
2,Hero MotoCorp Ltd.,Automobile and Auto Components,HEROMOTOCO,equity,2003-04-11
3,Mahindra & Mahindra Ltd.,Automobile and Auto Components,M&M,equity,1996-01-03
4,Maruti Suzuki India Ltd.,Automobile and Auto Components,MARUTI,equity,2003-07-09
5,Tata Motors Ltd.,Automobile and Auto Components,TATAMOTORS,equity,1998-07-22
6,Bharat Electronics Ltd.,Capital Goods,BEL,equity,2000-07-19
7,Larsen & Toubro Ltd.,Construction,LT,equity,2004-06-23
8,Grasim Industries Ltd.,Construction Materials,GRASIM,equity,1995-05-10
9,UltraTech Cement Ltd.,Construction Materials,ULTRACEMCO,equity,2004-08-24


In [4]:
# listing dates are all set to
new_rows = [
  {
    "company_name": "NIFTY 50",
    "industry": None,
    "symbol": "NIFTY 50",
    "listing_date": datetime.date(2014, 1, 1),
  },
  {
    "company_name": "NIFTY FIN SERVICE",
    "industry": "Financial Services",
    "symbol": "NIFTY FIN SERVICE",
    "listing_date": datetime.date(2014, 1, 1),
  },
  {
    "company_name": "NIFTY IT",
    "industry": "Information Technology",
    "symbol": "NIFTY IT",
    "listing_date": datetime.date(2014, 1, 1),
  },
  {
    "company_name": "NIFTY AUTO",
    "industry": "Automobile and Auto Components",
    "symbol": "NIFTY AUTO",
    "listing_date": datetime.date(2014, 1, 1),
  },
  {
    "company_name": "INDIA VIX",
    "industry": None,
    "symbol": "INDIA VIX",
    "listing_date": datetime.date(2014, 1, 1),
  },
]

df_new_rows = pd.DataFrame(new_rows).sort_values(by=["industry", "symbol"])
df_new_rows["type"] = "index"
df_all = pd.concat([df_new_rows, df_all], ignore_index=True).reset_index(drop=True)
df_all

,company_name,industry,symbol,listing_date,type
0,NIFTY AUTO,Automobile and Auto Components,NIFTY AUTO,2014-01-01,index
1,NIFTY FIN SERVICE,Financial Services,NIFTY FIN SERVICE,2014-01-01,index
2,NIFTY IT,Information Technology,NIFTY IT,2014-01-01,index
3,INDIA VIX,None,INDIA VIX,2014-01-01,index
4,NIFTY 50,None,NIFTY 50,2014-01-01,index
5,Bajaj Auto Ltd.,Automobile and Auto Components,BAJAJ-AUTO,2008-05-26,equity
6,Eicher Motors Ltd.,Automobile and Auto Components,EICHERMOT,2004-09-07,equity
7,Hero MotoCorp Ltd.,Automobile and Auto Components,HEROMOTOCO,2003-04-11,equity
8,Mahindra & Mahindra Ltd.,Automobile and Auto Components,M&M,1996-01-03,equity
9,Maruti Suzuki India Ltd.,Automobile and Auto Components,MARUTI,2003-07-09,equity


In [5]:
kite = cz.get_live_kite_object(force_refresh=False)
today = datetime.date.today()
rows = list(df_all.iterrows())
first_date_for_instrument_desc = {}
for _, row in tqdm.tqdm(rows):
  instrument_desc = (
    sid.EquityDesc(symbol=row["symbol"], market=sid.Market.INDIAN_MARKET)
    if row["type"] == "equity"
    else sid.IndexDesc(symbol=row["symbol"], market=sid.Market.INDIAN_MARKET)
  )
  df = uz.get_full_kite_data_for_range(
    instrument_desc=instrument_desc,
    date_range=sdr.DateRange(row["listing_date"], today),  # type: ignore
    kite=kite,
  )
  if not df["date"].is_unique:
    raise ValueError(f"Date is not unique for {instrument_desc}")
  if not df["date"].is_monotonic_increasing:
    raise ValueError(f"Date is not monotonic increasing for {instrument_desc}")
  path = udn.get_raw_path_from_instrument_desc(instrument_desc)
  os.makedirs(os.path.dirname(path), exist_ok=True)
  first_date_for_instrument_desc[instrument_desc] = df["date"].iloc[0].date()
  df.to_parquet(path)


100%|██████████| 55/55 [35:37<00:00, 38.85s/it]


In [7]:
df_all["first_date"] = df_all.apply(
  lambda x: first_date_for_instrument_desc[
    sid.EquityDesc(symbol=x["symbol"], market=sid.Market.INDIAN_MARKET)
    if x["type"] == "equity"
    else sid.IndexDesc(symbol=x["symbol"], market=sid.Market.INDIAN_MARKET)
  ],
  axis=1,
)
df_all

,company_name,industry,symbol,listing_date,type,first_date
0,NIFTY AUTO,Automobile and Auto Components,NIFTY AUTO,2014-01-01,index,2015-01-09
1,NIFTY FIN SERVICE,Financial Services,NIFTY FIN SERVICE,2014-01-01,index,2015-01-09
2,NIFTY IT,Information Technology,NIFTY IT,2014-01-01,index,2015-01-09
3,INDIA VIX,None,INDIA VIX,2014-01-01,index,2015-01-09
4,NIFTY 50,None,NIFTY 50,2014-01-01,index,2015-01-09
5,Bajaj Auto Ltd.,Automobile and Auto Components,BAJAJ-AUTO,2008-05-26,equity,2015-02-02
6,Eicher Motors Ltd.,Automobile and Auto Components,EICHERMOT,2004-09-07,equity,2015-02-02
7,Hero MotoCorp Ltd.,Automobile and Auto Components,HEROMOTOCO,2003-04-11,equity,2015-02-02
8,Mahindra & Mahindra Ltd.,Automobile and Auto Components,M&M,1996-01-03,equity,2015-02-02
9,Maruti Suzuki India Ltd.,Automobile and Auto Components,MARUTI,2003-07-09,equity,2015-02-02


In [9]:
df_all.to_csv(
  "/Users/chirayuagrawal/algotrading_v40/data/raw/indian_market/inventory.csv"
)